In [113]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import sys
sys.path.append('../src')

from math_utils import *

In [114]:
m = 10
n = 3
A = np.random.rand(m, n)
h, R = thin_QR(A)

In [116]:
def incr_QR(x_new:np.ndarray, householder_vectors:list, R:np.ndarray) -> tuple:
    '''
    Incremental QR decomposition of matrix X

    Parameters:
    -----------
    x_new: np.ndarray
        The new feature colunmn added to the original matrix
    householder_vectors: list
        Householder vectors from the previous QR factorization of X
    R: np.ndarray
        R matrix from the previous QR factorization of X

    Returns:
    --------
    tuple
        The updated list of Householder vectors and the new R matrix (only the upper triangular part)
    '''
    
    m, n = x_new.shape[0], len(householder_vectors)

    z = apply_householders_vector(householder_vectors, x_new, reverse=True)
    z0, z1 = z[:n], z[n:]

    # Compute the new Householder vector
    norm_z1 = np.linalg.norm(z1) 
    u_new = z1.copy()
    u_new[0] += np.sign(z1[0]) * norm_z1
    u_new /= np.linalg.norm(u_new) #TODO: controlla stabilità numerica (se A[i,i] è vicino a 0 trova soluzione)
    
    z1 -= 2*np.outer(u_new, u_new.T @ z1)

    return householder_vectors + [u_new], np.block([[R, z0], [np.zeros((m-n, n)), z1]])[:n+1, :]

In [117]:
# append column to A
x = np.random.randn(m, 1)
A = np.hstack((A, x))
h, R_new = incr_QR(x, h, R)
R_new = np.vstack((R_new, np.zeros((m-n-1, n+1))))

In [118]:
AA = apply_householders_matrix(h, R_new)

In [119]:
AA

array([[ 0.28143425,  0.00874097,  0.7985617 , -0.0159366 ],
       [ 0.41565585,  0.02328141,  0.79490851,  0.52524936],
       [ 0.68553976,  0.28198344,  0.59894979, -0.66923419],
       [ 0.23125184,  0.06241724,  0.82817136,  1.09968359],
       [ 0.81993133,  0.55247978,  0.58187991,  0.22369608],
       [ 0.77870439,  0.63958352,  0.8511441 , -0.08467809],
       [ 0.31236675,  0.0862417 ,  0.67998215,  1.16741251],
       [ 0.11598731,  0.80655436,  0.25531416,  0.10376153],
       [ 0.77348721,  0.86407319,  0.10582485,  1.61715615],
       [ 0.81928333,  0.08728759,  0.29084043, -1.29280479]])

In [120]:
A

array([[ 0.28143425,  0.00874097,  0.7985617 , -1.25236116],
       [ 0.41565585,  0.02328141,  0.79490851, -0.12995654],
       [ 0.68553976,  0.28198344,  0.59894979, -0.22127302],
       [ 0.23125184,  0.06241724,  0.82817136,  0.61491551],
       [ 0.81993133,  0.55247978,  0.58187991, -0.04999885],
       [ 0.77870439,  0.63958352,  0.8511441 , -0.51585589],
       [ 0.31236675,  0.0862417 ,  0.67998215,  0.77728881],
       [ 0.11598731,  0.80655436,  0.25531416,  0.01633507],
       [ 0.77348721,  0.86407319,  0.10582485,  1.65234186],
       [ 0.81928333,  0.08728759,  0.29084043, -1.42654153]])